In [9]:
import pandas as pd
import numpy as np
import json
import pickle
import seaborn as sns
from matplotlib import pyplot as plt
import random
from sklearn.model_selection import train_test_split
import anndata as ad
import scanpy as sc
import sys
import os
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import time
from sklearn.metrics import mean_squared_error
import scipy.stats
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from anndata import AnnData

In [2]:
pip install pandas matplotlib datashader bokeh holoviews scikit-image colorcet


Note: you may need to restart the kernel to use updated packages.


In [20]:
sys.path.append('/home/BS94_SUR/phD/review/models reproductibility/PriorVAE/github code')
import simpleVAE
import priorVAE
import utils
import Reconstruction_performance_metrics
from priorVAE import *
from simpleVAE import Encoder, Decoder, simpleVAE
from utils import EarlyStopping, LRScheduler
from Reconstruction_performance_metrics import calculate_sample_wise_correlation_numpy, calculate_overall_reconstruction_each_model, calculate_overall_reconstruction_each_model_detailed_info, input_reconstruction_pairwise_correlation_plot, input_reconstruction_pairwise_correlation_with_annotation_labels

In [10]:
def extract_x_y_from_adata(adata: AnnData, labels: pd.Series):
    X = pd.DataFrame(adata.X, index=adata.obs.index)
    y = adata.obs[labels]
    return X, y

def split_data(adata: AnnData, column_name: str, train_size: float = 0.8):
    X, y = extract_x_y_from_adata(adata, column_name)
    X_train,  X_test, labels_train,  labels_test = train_test_split(
        X, y, train_size=train_size, random_state=42, stratify=y)
    return X_train,  X_test, labels_train,  labels_test

def extract_index(X: np.ndarray):
    index_df = X.index
    return index_df

def build_adata_from_X(X: np.ndarray, adata: AnnData):
    index_df = extract_index(X)
    adata = adata[adata.obs.index.isin(index_df)]
    return adata, index_df

def encode_y(labels):
    le = preprocessing.LabelEncoder().fit(labels)
    y = torch.Tensor(le.transform(labels))
    return y

In [11]:
data_path = '/home/BS94_SUR/phD/review/models reproductibility/pmvae/scripts/data/kang_count.h5ad'
adata = sc.read_h5ad(data_path)
adata

AnnData object with n_obs × n_vars = 13576 × 979
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'stim', 'seurat_annotations', 'integrated_snn_res.0.5', 'seurat_clusters', 'condition', 'cell_type', 'INTERFERON_SIGNALING', 'INTERFERON_GAMMA_SIGNALING'
    uns: 'condition_colors', 'terms'
    obsm: 'X_latent'
    varm: 'I'

In [14]:
column_name = 'condition'
X, y = extract_x_y_from_adata(adata, column_name)
X_train,  X_test, y_train,  y_test = train_test_split(
    X, y, train_size=0.90, random_state=42, stratify=y)
print(df_train.shape)
print(df_test.shape)

(12218, 979)
(1358, 979)


In [16]:
n_features_trans = X_train.shape[1]
n_inputs = X_train.shape[1]
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [19]:
beta = 1
batch_size = 128
lr = 0.0001
epochs = 300
early_stopping = EarlyStopping()
criterion = nn.MSELoss(reduction='sum')
latent_dim = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
df_mean_direct = pd.DataFrame(columns=adata.var.index)
df_sigma_direct = pd.DataFrame(columns=adata.var.index)
df_mean_bootstrap = pd.DataFrame(columns=adata.var.index)
df_sigma_bootstrap = pd.DataFrame(columns=adata.var.index)
df_pvalues_mean = pd.DataFrame(columns=adata.var.index)

In [ ]:
train_set = torch.Tensor(X_train[:,0:n_features_trans])
test_set = torch.Tensor(X_test[:, 0:n_features_trans])
train_set = TensorDataset(train_set, train_set)
test_set = TensorDataset(test_set, test_set)
train_loader = DataLoader(
    train_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(
    test_set, batch_size=batch_size, shuffle=False)
model = priorVAE(n_inputs=n_inputs, n_features_path=, n_features_trans=nfeatures_trans).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
lr_scheduler = LRScheduler(optimizer)
print('my_xtest' + str(X_test.shape))